In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import utilities


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
tickers = pd.read_csv('../data_cleaning/stocks/dense_95.csv')

# for local testing
tickers = tickers.head(10)

In [13]:
# start with the A ticker

ticker = "AAL"
df = utilities.load_stock(ticker)

In [14]:
df.head()

,timestamp,open,high,low,close,volume,ticker,date,time,minute
0,2013-12-09 08:37:00,24.00,24.00,23.95,23.95,37500,AAL,2013-12-09,08:37:00,517
1,2013-12-09 08:38:00,23.75,23.75,23.75,23.75,1200,AAL,2013-12-09,08:38:00,518
2,2013-12-09 08:39:00,23.50,23.50,23.50,23.50,600,AAL,2013-12-09,08:39:00,519
3,2013-12-09 08:40:00,23.70,23.70,23.69,23.69,250,AAL,2013-12-09,08:40:00,520
4,2013-12-09 08:41:00,23.69,23.69,23.69,23.69,700,AAL,2013-12-09,08:41:00,521


In [15]:
df = utilities.RSI(df, 14)
df = utilities.RSI(df, 28)
df = utilities.RSI(df, 56)

df = utilities.MACD(df)

df = utilities.EMA(df, 10)
df = utilities.EMA(df, 50)
df = utilities.EMA(df, 200)

df = utilities.market_hours_only(df)

In [18]:
df[400:500]

              timestamp    open    high    low   close  volume ticker  \
595 2013-12-10 09:39:00  24.630  24.690  24.57  24.670   97230    AAL   
596 2013-12-10 09:40:00  24.680  24.890  24.68  24.880  159276    AAL   
597 2013-12-10 09:41:00  24.890  24.930  24.83  24.930  111074    AAL   
598 2013-12-10 09:42:00  24.930  24.955  24.87  24.930  193432    AAL   
599 2013-12-10 09:43:00  24.910  24.930  24.75  24.790  144983    AAL   
..                  ...     ...     ...    ...     ...     ...    ...   
690 2013-12-10 11:14:00  24.755  24.755  24.71  24.730   43861    AAL   
691 2013-12-10 11:15:00  24.720  24.740  24.72  24.720   24640    AAL   
692 2013-12-10 11:16:00  24.720  24.730  24.72  24.720   26926    AAL   
693 2013-12-10 11:17:00  24.720  24.730  24.72  24.725   78718    AAL   
694 2013-12-10 11:18:00  24.720  24.740  24.70  24.720  119730    AAL   

           date      time  minute    rsi14    rsi28    rsi56    macd  signal  \
595  2013-12-10  09:39:00     579  48.0000 